In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说吧
### 测试看下来，单纯得跑模型似乎会应该Y值的方差而产生巨大问题，我们需要重新做一次分类，明天的核心任务吧

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
from param_config import config

In [2]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']


###清理异常Y值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]
dfTrain = dfTrain.reset_index(drop=True)

#dfTrain = dfTrain.loc[dfTrain['Y']<=dfTrain['Y'].quantile(0.9)]

In [3]:
def FeatAll(train,pred,pcentByother=False,rankMinus=False,missing=False):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days
    
    ###RF填补缺失值
    dfAll = fe.fillna_RF(dfAll,predictors,True)
       
    
    ###尝试处理异常值
    for var in predictors:
        dfAll.loc[dfAll[var]>=dfAll[var].quantile(0.95)*3,var] = dfAll[var].quantile(0.95)*3
        #dfAll.loc[dfAll[var]>=df[var].quantile(0.99)*1.5,var] = np.nan

    ###这个部分将每一列分箱，并且计算基于该列分类的其他列的百分比顺序
    if pcentByother:
        varDict = {}
        bin_set = 15
        for colBase in predictors+['sex','age']:
            if float(dfAll[colBase].isnull().sum())/dfAll.shape[0]>=0.4:
                print('Too many missing in variable %s'%colBase)
                continue
            if dfAll[colBase].nunique()<=5:
                varDict[colBase] = [x for x in predictors if x!=colBase]
            else:
                dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=bin_set,labels=[colBase+'_bin_%d'%i for i in range(bin_set)])
                dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(str)
                dfAll[colBase+'_based'].fillna(colBase+'_bin_NA',inplace = True)
                varDict[colBase+'_based'] = [x for x in predictors if x!=colBase]
        try: 
            dfAllPcentByOther = pd.read_csv('../../Cache/pcent_by_other_bin%d.csv'%bin_set)
        except:
            print('There is no Pcent by other col, will generate and save')
            dfAllPcentByOther = fe.pcent_by_other_col(dfAll,varDict,['ID'])
            dfAllPcentByOther.to_csv('../../Cache/pcent_by_other_bin%d.csv'%bin_set,index=False)
        for var in dfAll.columns:
            if '_based' in var:
                del dfAll[var]
                
    if rankMinus:
        varList = []
        bin_set = 30
        if bin_set >= dfAll.shape[0]:
            for colBase in predictors:
                dfAll[colBase+'_based'] = dfAll[colBase].rank(method='max')/float(dfAll.shapoe[0])
                varList.append(colBase+'_based')
        else:
            for colBase in predictors:
                dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=bin_set,labels=range(bin_set))
                dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(int)
                varList.append(colBase+'_based')
        try: 
            dfrankMinus = pd.read_csv('../../Cache/rankMinus_bin%d.csv'%bin_set)
        except:
            print('There is no Rank Minus, will generate and save')
            dfrankMinus = fe.var_minus(dfAll,varList,['ID'],0)
            dfrankMinus.to_csv('../../Cache/rankMinus_bin%d.csv'%bin_set,index=False)
            
    ###根据日期放置一个百分比顺序作为尝试
    dfAllPcentByDate = fe.pcent_by_other_col(dfAll,{'date':predictors},['ID']) 
    
    '''dfAll['date'] = pd.qcut(dfAll['date'],q=10,labels=['date_bin_%d'%i for i in range(10)])
    dfAll = pd.concat([dfAll,pd.get_dummies(dfAll['date'])],axis=1)'''
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in
    
    for plus in [['PartII_1','PartII_2'],['PartIII_1','PartIII_2','PartIII_3']]:
        tmpVar =plus[0]
        tmpValue = dfAll[plus[0]].values
        for var in plus[1:]:
            tmpVar = tmpVar +'_plus_' + var
            tmpValue = tmpValue + dfAll[var].values
        dfAll[tmpVar] = tmpValue
    
    dfAll['PartI_9'] = dfAll['PartI_5']-dfAll['PartI_6']-dfAll['PartI_7']
    dfAll['PartII_5'] = dfAll['PartII_2']-dfAll['PartII_3']-dfAll['PartII_4']
    
    for ratio in [['PartII_1','PartII_2'],['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]
        
    for multiply in [['PartI_1','PartI_5'],['PartI_2','PartI_5'],['PartI_3','PartI_5'],['PartI_4','PartI_5'],['PartVI_1','PartVI_2'],['PartV_2','PartV_5'],['PartV_2','PartV_7'],['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]*dfAll[multiply[1]]
    
    
    ###看看missing 的情况
    '''dfAll['missing_cnt'] = dfAll[check_missing].isnull().sum(axis=1)
    for i in range(len(check_missing)):
        dfAll['missing_part%d'%i] = dfAll[check_missing[i]].isnull().astype(int)'''
    
    
    '''for key,group in {'PartI':['PartI_1','PartI_2','PartI_3','PartI_4'],'PartII':['PartII_1','PartII_3','PartII_4']}.items():
        dfAll['group_'+key+'_std'] = dfAll[group].std(axis=1)
        dfAll['group_'+key+'_mean'] = dfAll[group].mean(axis=1)
        dfAll['group_'+key+'_median'] = dfAll[group].median(axis=1)'''
    
    
    try:
        dfAll = dfAll.merge(dfAllPcentByOther,'inner','ID')
    except:
        print('No Pcent by other col')
        
    try:
        dfAll = dfAll.merge(dfrankMinus,'inner','ID')
    except:
        print('No Rank Minus')
        
    
    dfAll = dfAll.merge(dfAllPcentByDate,'inner','ID')
    
    
    ###填补缺失值
    if missing:
        dfAll =dfAll.fillna(dfAll.median())
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred

In [4]:
dfTrain,dfPred = FeatAll(dfTrain,dfPred,False,False,False)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


No Pcent by other col
No Rank Minus


In [5]:
pd.set_option('display.max_rows',len(predictors))
len(predictors)


98

In [47]:
predictorsRF = fs.RF_selection(dfTrain,predictors)

In [8]:
modelPredictor = predictors
#modelPredictor = predictorsRF[:50]

In [50]:
n_splits=5
early_stop=50
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,'colsample_bytree':0.8}
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,modelPredictor,n_splits,early_stop=early_stop,params=params)

No metric defined, will use rmse in the model
[0]	train-rmse:5.27711	eval-rmse:5.32352
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.24588	eval-rmse:2.39078
[200]	train-rmse:1.31665	eval-rmse:1.60125
[300]	train-rmse:1.06954	eval-rmse:1.45215
[400]	train-rmse:0.983299	eval-rmse:1.42606
[500]	train-rmse:0.927499	eval-rmse:1.41718
Stopping. Best iteration:
[520]	train-rmse:0.918793	eval-rmse:1.41678

Best tree is 521, performance is 0.918793, 1.416778
No metric defined, will use rmse in the model
[0]	train-rmse:5.29704	eval-rmse:5.24311
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.26499	eval-rmse:2.26167
[200]	train-rmse:1.33534	eval-rmse:1.44537
[300]	train-rmse:1.08375	eval-rmse:1.29754
[400]	train-rmse:0.993188	eval-rmse:1.27819
[500]	train-rmse:0.938721	e

In [51]:
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

In [52]:
tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:30]
imp.sort_values('sum_imp',ascending=False).reset_index()

,index,lk,variable,imp_fold1,imp_fold2,imp_fold3,imp_fold4,imp_fold5,sum_imp
0,0,f0,age,0.098594,0.113608,0.105247,0.095564,0.096138,0.509152
1,1,f1,PartII_1,0.049880,0.037565,0.047633,0.042847,0.047825,0.225750
2,15,f15,PartV_2_multiply_PartV_7,0.031777,0.034912,0.044155,0.039422,0.039514,0.189780
3,11,f11,PartI_1,0.036516,0.035575,0.036595,0.025430,0.032997,0.167112
4,3,f3,PartII_3_divided_PartII_2,0.026416,0.033336,0.030319,0.035938,0.035930,0.161939
5,17,f17,PartIII_3,0.033253,0.024878,0.027219,0.033674,0.032997,0.152020
6,2,f2,PartII_1_plus_PartII_2,0.033564,0.033585,0.029639,0.026184,0.025257,0.148228
7,41,f41,PartV_1_multiply_PartV_10,0.021444,0.029687,0.029790,0.031294,0.023546,0.135760
8,6,f6,PartI_4_multiply_PartI_5,0.029291,0.022970,0.029185,0.025081,0.028516,0.135043
9,27,f27,PartIII_1,0.023075,0.026785,0.024119,0.030771,0.023790,0.128540


In [53]:
test_result,result,imp_selected = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop=early_stop,params=params)

No metric defined, will use rmse in the model
[0]	train-rmse:5.27693	eval-rmse:5.32346
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.24526	eval-rmse:2.3896
[200]	train-rmse:1.31655	eval-rmse:1.59794
[300]	train-rmse:1.07564	eval-rmse:1.44725
[400]	train-rmse:0.996267	eval-rmse:1.41725
[500]	train-rmse:0.942722	eval-rmse:1.41083
[600]	train-rmse:0.903782	eval-rmse:1.40869
[700]	train-rmse:0.869505	eval-rmse:1.40796
Stopping. Best iteration:
[722]	train-rmse:0.862858	eval-rmse:1.40722

Best tree is 723, performance is 0.862858, 1.407216
No metric defined, will use rmse in the model
[0]	train-rmse:5.29707	eval-rmse:5.24288
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:2.2672	eval-rmse:2.2641
[200]	train-rmse:1.3398	eval-rmse:1.43975
[300]	train-rmse:1.09967	eval-

In [54]:
for i in range(1,n_splits+1):
    imp_selected['imp_fold%d'%i] = imp_selected['imp_fold%d'%i]/imp_selected['imp_fold%d'%i].sum()
imp_selected['sum_imp'] = imp_selected[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)
imp_selected.sort_values('sum_imp',ascending=False).reset_index()

,index,lk,variable,imp_fold1,imp_fold2,imp_fold3,imp_fold4,imp_fold5,sum_imp
0,0,f0,age,0.106690,0.115442,0.109465,0.098161,0.099271,0.529029
1,2,f2,PartV_2_multiply_PartV_7,0.067767,0.065689,0.075562,0.066289,0.072569,0.347877
2,1,f1,PartII_1,0.050681,0.047060,0.049678,0.045359,0.051192,0.243969
3,3,f3,PartI_1,0.046742,0.048182,0.053077,0.035626,0.047424,0.231050
4,7,f7,PartV_1_multiply_PartV_10,0.037069,0.041673,0.041747,0.039824,0.035957,0.196270
5,5,f5,PartIII_3,0.038633,0.031199,0.034687,0.042496,0.040708,0.187723
6,11,f11,pcent_date_PartV_7,0.036664,0.037259,0.032421,0.039379,0.040953,0.186676
7,4,f4,PartII_3_divided_PartII_2,0.032493,0.033368,0.032770,0.044023,0.043820,0.186475
8,23,f23,PartV_2_multiply_PartV_5,0.035332,0.036211,0.032247,0.035371,0.037022,0.176183
9,9,f9,PartIII_1,0.029482,0.036436,0.031550,0.038679,0.031370,0.167517


In [55]:
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
print((result['score']>6.8).sum())
print(result['score'].min())
print(result['score'].max())
print(result['score'].mean())
print(result['score'].median())
print("-------------------------")
print(result.iloc[938,:])

49
4.827
10.3291
5.644790172576904
5.519465446472168
-------------------------
ID         6705.000000
Score_1      10.974416
Score_2       9.732619
Score_3       9.572588
Score_4      11.810825
Score_5       9.555130
score        10.329115
Name: 938, dtype: float64


In [ ]:
dfTrain['Y'].mean()

# 基础变量
1.83977998784

###增加部分比例数据，已经缺失体检项目情况
1.84295123211  imp来看，missing信息没有意义

###上一版去除missing
1.83839589522



###增加一些比值信息
1.84354856365


###根据日期的百分比，效果更加不好了
1.85046680299  变量选择之后 1.83862284729



###前面所有的特征
1.86013896413   变量选择之后  1.85446185976
更改了一下模型的参数 1.81447588639  变量选择之后  1.80873726211
用中位数填补缺失值 1.81136029536  变量选择之后  1.80584671605
将异常值取空 然后按中位数填补 1.83149451674  变量选择之后  1.82105909448
将异常值cap 然后按中位数填补 1.80727635099  变量选择之后  1.81587223984
根据上一版用均值填补缺失（已有cap）  1.81374808226 ///   1.8175380386


###增加更多特征，主要涉及加减乘除
用中位数填补缺失值 1.81045571166 ///  1.78855151825  实际效果极差
不弥补缺失值  1.80566299608  ///  1.79592827524


####增加各列排行相减的变量，中位数填缺失，

In [22]:
other_note ='_RFSelection'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)

In [ ]:
###NN

# NN

In [6]:
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.layers.recurrent import LSTM,GRU
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
# initializes weights with N(0,0.1)
from keras import initializers
from keras.optimizers import Adam
import keras
from keras.utils import np_utils
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

Using TensorFlow backend.


In [9]:
scale = StandardScaler()
dfTrain.loc[:,modelPredictor] = scale.fit_transform(dfTrain.loc[:,modelPredictor])
dfPred.loc[:,modelPredictor] = scale.transform(dfPred.loc[:,modelPredictor])

featNum = len(modelPredictor)

In [10]:
model = Sequential()
model.add(Dense(int(featNum*1.5),activation='tanh',input_shape=(featNum,),kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(featNum,activation='tanh',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(int(featNum/2),activation='tanh',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
model.add(Dense(int(featNum/4),activation='tanh',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))
'''model.add(Dense(int(featNum/8),activation='relu',kernel_initializer='glorot_normal'))
model.add(Dropout(0.4))'''
model.add(Dense(1,activation='linear'))
model.compile(optimizer='adam',loss='mse',metrics=['mae'])

In [11]:
model.fit(dfTrain[modelPredictor].values,dfTrain['Y'].values,epochs=100,batch_size=5,verbose=2,validation_split=0.2)

Train on 4512 samples, validate on 1129 samples
Epoch 1/100
4s - loss: 6.9321 - mean_absolute_error: 1.8442 - val_loss: 2.3508 - val_mean_absolute_error: 0.8717
Epoch 2/100
3s - loss: 3.0607 - mean_absolute_error: 1.1676 - val_loss: 2.3049 - val_mean_absolute_error: 0.8266
Epoch 3/100
3s - loss: 2.9602 - mean_absolute_error: 1.1030 - val_loss: 2.3275 - val_mean_absolute_error: 0.8028
Epoch 4/100
3s - loss: 2.8002 - mean_absolute_error: 1.0775 - val_loss: 2.3117 - val_mean_absolute_error: 0.7447
Epoch 5/100
3s - loss: 2.6863 - mean_absolute_error: 1.0458 - val_loss: 2.2557 - val_mean_absolute_error: 0.7635
Epoch 6/100
3s - loss: 2.6942 - mean_absolute_error: 1.0537 - val_loss: 2.2990 - val_mean_absolute_error: 0.7460
Epoch 7/100
3s - loss: 2.5999 - mean_absolute_error: 1.0245 - val_loss: 2.2261 - val_mean_absolute_error: 0.7495
Epoch 8/100
3s - loss: 2.5555 - mean_absolute_error: 1.0024 - val_loss: 2.2794 - val_mean_absolute_error: 0.7518
Epoch 9/100
3s - loss: 2.5109 - mean_absolute_er

3s - loss: 1.2486 - mean_absolute_error: 0.6921 - val_loss: 2.1473 - val_mean_absolute_error: 0.7777
Epoch 74/100
4s - loss: 1.2569 - mean_absolute_error: 0.6904 - val_loss: 2.4671 - val_mean_absolute_error: 0.8452
Epoch 75/100
3s - loss: 1.1939 - mean_absolute_error: 0.6932 - val_loss: 2.2002 - val_mean_absolute_error: 0.7580
Epoch 76/100
3s - loss: 1.2227 - mean_absolute_error: 0.6917 - val_loss: 2.2281 - val_mean_absolute_error: 0.7734
Epoch 77/100
3s - loss: 1.1790 - mean_absolute_error: 0.6849 - val_loss: 2.1922 - val_mean_absolute_error: 0.7620
Epoch 78/100
3s - loss: 1.1514 - mean_absolute_error: 0.6864 - val_loss: 2.3152 - val_mean_absolute_error: 0.7795
Epoch 79/100
3s - loss: 1.1636 - mean_absolute_error: 0.6875 - val_loss: 2.3736 - val_mean_absolute_error: 0.8011
Epoch 80/100
3s - loss: 1.1308 - mean_absolute_error: 0.6760 - val_loss: 2.2241 - val_mean_absolute_error: 0.7601
Epoch 81/100
3s - loss: 1.1660 - mean_absolute_error: 0.6803 - val_loss: 2.2249 - val_mean_absolute_e

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 147)               14553     
_________________________________________________________________
dropout_1 (Dropout)          (None, 147)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 98)                14504     
_________________________________________________________________
dropout_2 (Dropout)          (None, 98)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 49)                4851      
_________________________________________________________________
dropout_3 (Dropout)          (None, 49)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                1200      
__________

In [23]:
len(model.layers)/2

4.5